In [1]:
REQUEST_ID = 265
AOI = 'POLYGON ((-94.783452 42.110879, -94.77976099999999 42.110799, -94.779718 42.117867, -94.78068399999999 42.117898, -94.78089799999999 42.117676, -94.781499 42.117691, -94.78179900000001 42.117755, -94.781756 42.118026, -94.783258 42.118073, -94.783452 42.117978, -94.783452 42.115129, -94.783452 42.110879))'
START_DATE = '2020-07-07'
END_DATE = '2020-07-14'

In [2]:
# !pip3 install -q tslearn

In [3]:
import os
import shutil
import json
import time
import cv2
import rasterio
import pandas as pd
import numpy as np
import geopandas as gpd
import rasterio.mask
import shapely

from tqdm.notebook import tqdm
from os.path import join, basename, split
from scipy.ndimage import rotate
from rasterio import features
from rasterio.merge import merge
from rasterio.warp import (aligned_target,
                           calculate_default_transform,
                           reproject,
                           Resampling)
from shapely.geometry import Polygon, shape, LinearRing, box
import shapely.wkt
from pathlib import Path
from datetime import datetime
import matplotlib.pylab as plt
import seaborn as sns

#from tslearn.clustering import TimeSeriesKMeans
#from tslearn.utils import to_time_series_dataset
import rasterio.mask as riomask

from code.download.utils import get_tiles, _check_folder, check_nodata, get_min_clouds, transform_resolution
from code.download.load_tiles import load_images
from code.index_research import combine_bands, calculate_ndvi, calculate_tcari, calculate_msr_g
from code.plant_stress import PlantStress

import warnings
warnings.filterwarnings('ignore')
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [4]:
default_crs = 'EPSG:4326'

polygon = shapely.wkt.loads(AOI)
aoi_filename = f"{time.time()}_aoi.geojson"
gpd.GeoDataFrame(gpd.GeoSeries([polygon]), columns=["geometry"]).to_file(aoi_filename, driver="GeoJSON")

In [5]:
SEED = 66
NB_USER = os.getenv('NB_USER')
BASE = f"/home/{NB_USER}/work"

API_KEY = os.path.join(BASE, ".secret/sentinel2_google_api_key.json")
sentinel_tiles_path = os.path.join(BASE, "notebooks/pbdnn/sentinel2grid.geojson")
LOAD_DIR = os.path.join(BASE, "satellite_imagery")
RESULTS_DIR = os.path.join(BASE, "results/pbdnn")
PBD_DIR = os.path.join(BASE, "notebooks/pbdnn")

BANDS = {'B04','B08', 'TCI'}
CONSTRAINTS = {'NODATA_PIXEL_PERCENTAGE': 15.0, 'CLOUDY_PIXEL_PERCENTAGE': 10.0, }
PRODUCT_TYPE = 'L2A'

In [6]:
b04_tiles, b08_tiles, tci_tiles = [], [], []
date_tile_info = get_tiles(aoi_filename, sentinel_tiles_path)

In [7]:
loadings = load_images(API_KEY, date_tile_info.tileID.values, START_DATE, END_DATE, LOAD_DIR, BANDS, CONSTRAINTS, PRODUCT_TYPE)
checked = check_nodata(loadings, PRODUCT_TYPE)

try:
    checked = get_min_clouds(checked)
except Exception as e:
    print(f'No clean raster found for period from {start} to {end}, skipping, {e}')

for i, tile in date_tile_info.iterrows():
    try:
        tile_folder = Path(checked[tile.tileID])
        print(f'filtered: {tile_folder}')
    except Exception as ex:
        print(ex)

b04_tile = [os.path.join(tile_folder, filename) for filename in os.listdir(tile_folder) if 'B04_10m.jp2' in filename][0]
b08_tile = [os.path.join(tile_folder, filename) for filename in os.listdir(tile_folder) if 'B08_10m.jp2' in filename][0]
tci_tile = [os.path.join(tile_folder, filename) for filename in os.listdir(tile_folder) if 'TCI_10m.jp2' in filename][0]

filtered: /home/jovyan/work/satellite_imagery/S2A_MSIL2A_20200713T170901_N0214_R112_T15TUG_20200713T213310


In [8]:
b04_tile

'/home/jovyan/work/satellite_imagery/S2A_MSIL2A_20200713T170901_N0214_R112_T15TUG_20200713T213310/T15TUG_20200713T170901_B04_10m.jp2'

In [9]:
os.makedirs(f'{BASE}/results/dd/plant_stress/{REQUEST_ID}', exist_ok=True)
ndvi_path = f'{BASE}/results/dd/plant_stress/{REQUEST_ID}/{REQUEST_ID}_ndvi.tif'
if not os.path.exists(ndvi_path):
    calculate_ndvi(b04_tile, b08_tile, out_path=ndvi_path, nodata=np.nan)

In [10]:
min_ndvi = 0.3
z_score = 5
z_score_anom = 1
colors = {"Normal Growth": (0, 0, 0), "Anomaly": (182, 10, 28)}

field = gpd.read_file(aoi_filename)

In [11]:
ps = PlantStress(min_ndvi=min_ndvi, noise_z_score=z_score, anomaly_z_score=z_score_anom)

In [12]:
ps.segment_field(field, ndvi_path, START_DATE, END_DATE, REQUEST_ID)

'/home/jovyan/work/results/dd/plant_stress/265/265_field_265_0.tif'

In [13]:
remove_aux_rasters = True

if remove_aux_rasters:
    
    try:
        shutil.rmtree(tile_folder)
    except FileNotFoundError:
        print(f'Error removing directory {tile_folder}')
        
    try:
        os.remove(ndvi_path)
    except FileNotFoundError:
        print(f'File {ndvi_path} not found')
        
    try:
        os.remove(aoi_filename)
    except FileNotFoundError:
        print(f'File {aoi_filename} not found')